# Extração de Dados

Código tem o objetivo de extrarir os dados de Tuberculose do siste do Ministério da Saúde (DataSus).

Link do Site: http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sinannet/cnv/tubercbr.def

### Imports

In [ ]:
from selenium import webdriver
from selenium.webdriver import ActionChains, ChromeOptions
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common import keys
from bs4 import BeautifulSoup
import time
import pandas as pd

### Extração dos dados

In [ ]:
# Função do Selenium para realizar as seleções no site do DataSUS

def safe_click(xpath, driver, max_retry = 3):
    try:
        to_click = driver.find_element_by_xpath(xpath)
        to_click.click()
    except:
        print("ERRO NO CLICK")
        if max_retry != 0:
            time.sleep(10)
            safe_click(xpath, driver, max_retry-1)
        else:
            print('NUMERO DE TENTATIVAS EXCEDIDA')
            get_current_data()
            raise

In [ ]:
# Função para executar as seleções no site do DataSUS, por mês e considerando os filtros selecionados

lista_mes=['Janeiro','Fevereiro','Marco','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro']

df_datasus = pd.DataFrame() # Criação do dataframe final com os dados brutos

for mes in lista_mes: # Iteração por mês
    options = ChromeOptions()
    options.add_argument('headless') # Opção para não mostrar a tela durante a execução
    driver = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver', options=options) # Driver do chrome
    driver.get('http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sinannet/cnv/tubercbr.def') # Acesso ao site do DataSUS
    for ano in list(range(2001, 2019)): # Seleção dos anos disponíveis no site
        safe_click(f'//div[@class="periodo"]//option[contains(.,"{ano}")]', driver)
        time.sleep(1)
    # Seleção da forma de apresentar os dados (por ano de diagnóstico e UF)
    safe_click('//option[@value="UF_de_notificação"]', driver) 
    safe_click('(//option[@value="Ano_Diagnóstico"])[2]', driver) 
    safe_click('(//img[@id="fig2"])', driver)
    # Seleção do mês da iteração
    safe_click('//*[@name="SMês_Diagnóstico"]/option[contains(.,"Todas as categorias")]', driver)
    safe_click(f'//*[@name="SMês_Diagnóstico"]/option[contains(.,"{mes}")]', driver)
    safe_click('(//img[@id="fig43"])', driver)
    time.sleep(1)
    # Tipos de Entrada (Selecionar: "Caso Novo", "Não Sabe" e "Pós Óbito")
    safe_click('//*[@name="STipo_de_entrada__"]/option[contains(.,"CASO NOVO")]', driver)
    time.sleep(1)
    safe_click('//*[@name="STipo_de_entrada__"]/option[contains(.,"NÃO SABE")]', driver)
    time.sleep(1)
    safe_click('//*[@name="STipo_de_entrada__"]/option[contains(.,"PÓS ÓBITO")]', driver)
    time.sleep(1)
    # Retirar a opção "Todas as categorias da seleção (já vem selecionado)"
    safe_click('//*[@name="STipo_de_entrada__"]/option[contains(.,"Todas as categorias")]', driver)

    safe_click('//*[@value="prn"]', driver)
    safe_click('//input[@class="mostra"]', driver)
    
    dados = driver.find_element_by_xpath('//div[@name="governo"]')
    html = dados.get_attribute("innerHTML")
    soup = BeautifulSoup(html, "html.parser") 
    #mydivs = soup.findAll('text') 
    mydivs = soup.findAll("pre") 
    data = []
    for div in mydivs:
        data.append(div.text)


    lale = (data[0].split('\n'))
    # Inserindo os dados no dataframe final
    lista_dados = []
    for i in list(range(0, 28)):
        lista_dados.append(lale[i].split(';'))

    print(f'Finished {mes}')
    df_mes = pd.DataFrame(lista_dados[1:], columns=lista_dados[0])
    df_mes['mes'] = mes
    df_datasus = pd.concat([df_datasus, df_mes])

### Dados brutos

In [ ]:
df_datasus

### Exportação da Base Bruta

In [ ]:
df_datasus.to_csv('df_datasus.csv',sep=';',index=False,encoding='utf-8-sig')